In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

In [2]:
DeltaTable.createIfNotExists(spark)\
          .tableName("Gold_OrderPriority")\
          .addColumn("OrderPriority_ID",LongType())\
          .addColumn("Order_Priority",StringType())\
          .addColumn("Created_TS",TimestampType())\
          .addColumn("Modified_TS",TimestampType())\
          .execute()

StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 4, Finished, Available, Finished)

In [3]:
df=spark.read.table("Project_Lakehouse.gold_orderpriority")

StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 5, Finished, Available, Finished)

In [4]:
Max_Date=df.selectExpr("coalesce(max(Modified_TS),'1900-01-01')").first()[0]

StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 6, Finished, Available, Finished)

In [5]:
Max_Date

StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 7, Finished, Available, Finished)

'1900-01-01'

In [7]:
df_bronze=spark.read.table("Project_Lakehouse.bronze_sales")

StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 9, Finished, Available, Finished)

In [13]:
df_bronze_mod=df_bronze.select("Order_Priority").where(col("Modified_TS")>Max_Date).drop_duplicates()

StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 15, Finished, Available, Finished)

In [9]:
Max_ID=df.selectExpr("coalesce(max(OrderPriority_ID),0)").first()[0]

df_final=df_bronze_mod.withColumn("OrderPriority_ID",Max_ID+monotonically_increasing_id()+1)

StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 11, Finished, Available, Finished)

In [10]:
df_gold_delta=DeltaTable.forPath(spark,"Tables/gold_orderpriority")
df_bonze_table=df_final

StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 12, Finished, Available, Finished)

In [11]:
df_gold_delta.alias("gold")\
             .merge(\
               df_bonze_table.alias("bronze"),\
                "gold.Order_Priority==Bronze.Order_Priority"\
             )\
             .whenMatchedUpdate(\
                  set={
                    "gold.Modified_TS":current_timestamp()
                  }
             )\
             .whenNotMatchedInsert(\
                    values={
                         "gold.OrderPriority_ID":"Bronze.OrderPriority_ID",
                     "gold.Order_Priority":"Bronze.Order_Priority",
                     "gold.Created_TS":current_timestamp(),
                     "gold.Modified_TS":current_timestamp()
                    }
             )\
             .execute()


StatementMeta(, 804cdced-9fe6-4006-aea0-ae0e432829de, 13, Finished, Available, Finished)